# **Recommendation GUI screen**

### Meal type and input ingredients are used to recommend top 10 recipes based on recommendation score.
### Co-occurring ingredients can be found out using the co-occurring ingredient search button.
### Any of the recommended recipe can be further clicked to get more information about it.

***Created by Rahul Maheshwari***

In [ ]:
#! /usr/bin/env python
#  -*- coding: utf-8 -*-
import ast
import pickle
import sys
from tkinter import END, SINGLE
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
import pattern as pt
import recipe_info as ri

**Invoking the GUI Screen components**

In [ ]:
try:
    import Tkinter as tk
except ImportError:
    import tkinter as tk

try:
    import ttk

    py3 = False
except ImportError:
    import tkinter.ttk as ttk

    py3 = True

import recommendation_support


def vp_start_gui():
    '''Starting point when module is the main routine.'''
    global val, w, root
    root = tk.Tk()
    recommendation_support.set_Tk_var()
    top = Toplevel1(root)
    recommendation_support.init(root, top)
    root.mainloop()


w = None


def create_Toplevel1(rt, *args, **kwargs):
    '''Starting point when module is imported by another module.
       Correct form of call: 'create_Toplevel1(root, *args, **kwargs)' .'''
    global w, w_win, root
    # rt = root
    root = rt
    w = tk.Toplevel(root)
    recommendation_support.set_Tk_var()
    top = Toplevel1(w)
    recommendation_support.init(w, top, *args, **kwargs)
    return (w, top)


def destroy_Toplevel1():
    global w
    w.destroy()
    w = None


def suggest_screen():
    pt.vp_start_gui()

* **Method to calculate recommendation score based on the ingredients and meal type selected by the user.**

* **Normalized Numerical features are used in order to calculate the recommendation score for each recipe of specified meal type and then sorted based on the score and top 10 are shown in the UI.**

In [ ]:
def recommend(data_type_recipes, ingredients, k):
    com_val = {}
    for i in range(len(data_type_recipes)):
        neg_factors = np.mean([data_type_recipes['Calories'].iloc[i], data_type_recipes['Fat'].iloc[i],
                               data_type_recipes['Cholesterol'].iloc[i]])
        pos_factors = np.mean([data_type_recipes['Rating Score'].iloc[i], data_type_recipes['Carbs'].iloc[i],
                               data_type_recipes['Fiber'].iloc[i], data_type_recipes['Protein'].iloc[i]])
        com_val[data_type_recipes['Recipe ID'].iloc[i]] = len(
            set(ingredients) & set(data_type_recipes['Lookup Ingredients'].iloc[i])) * (pos_factors - neg_factors)
    return sorted(com_val, key=com_val.get, reverse=True)[:k]

## **Class containing methods to preprocess ingredients, calculate recommendation score, populating the list box with the recommended recipe title which can further be clicked to get complete information about the required recipe.**

In [ ]:
class Toplevel1:

    def recommend_recipes(self):
        k=10
        lm = WordNetLemmatizer()
        ingredients = str(self.Entry1.get()).split(',')
        ingredients = [i.strip() for i in ingredients]
        ingredients = [lm.lemmatize(i) for i in ingredients]
        meal_type = str(self.TCombobox1.get())

        lmtzr = WordNetLemmatizer()
        # Preprocessing
        data = pd.read_csv("recipes.csv")
        data2 = data.copy()
        temp = []
        val = 1.0
        for y in data['Calories']:
            try:
                val = float(y)
            except:
                y = y.replace(",", "")
                val = float(y)
            temp.append(val)
        data['Calories'] = temp
        cols = ['Ingredients', 'Cooking instructions', 'Rating', 'Lookup Ingredients']
        for x in cols:
            l = []
            for y in data[x]:
                try:
                    res = ast.literal_eval(y)
                    if x == 'Lookup Ingredients':
                        res = [lmtzr.lemmatize(word) for word in res]
                except:
                    print(y)
                l.append(res)
            data[x] = l
        cols_to_norm = ['Calories', 'Fat', 'Cholesterol', 'Carbs', 'Fiber', 'Protein']
        data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
        data_type_recipes = data[data['Meal'] == meal_type]
        data_type_recipes = data_type_recipes.dropna()
        recommendations = recommend(data_type_recipes, ingredients, k)
        recommendation_urls = []
        recommend_recipes = []
        for x in recommendations:
            val = data_type_recipes.loc[data_type_recipes['Recipe ID'] == x, ['URL', 'Title']]
            recommendation_urls.append(list(val['URL'])[0])
            recommend_recipes.append(list(val['Title'])[0])
        self.Scrolledlistbox1.insert(END, "")
        self.Scrolledlistbox1.insert(END, "Top 10 Recommended recipes for " + str(meal_type))
        self.Scrolledlistbox1.insert(END, "")
        if len(recommend_recipes) != 0:
            for r in recommend_recipes:
                self.Scrolledlistbox1.insert(END, str(r))
        else:
            self.Scrolledlistbox1.insert(END, str("No recipes for given ingredients"))

    def suggest(self):
        lm = WordNetLemmatizer()
        ingredients = str(self.Entry1.get()).split(',')
        ingredients = [i.strip() for i in ingredients]
        ingredients = [lm.lemmatize(i) for i in ingredients]
        meal_type = str(self.TCombobox1.get())
        dbfile = open('ingredients_pickle', 'wb')
        pickle.dump(ingredients, dbfile)
        dbfile.close()
        dbfile = open('meal_type_pickle', 'wb')
        pickle.dump(meal_type, dbfile)
        dbfile.close()
        suggest_screen()

    def list_item_select(self, event):
        selected_recipe_name = self.Scrolledlistbox1.get(self.Scrolledlistbox1.curselection())
        dbfile = open('selected_recipe_name', 'wb')
        pickle.dump(selected_recipe_name, dbfile)
        dbfile.close()
        ri.vp_start_gui()
        # webbrowser.open_new("https://www.kaggle.com/")

    def __init__(self, top=None):
        """This class configures and populates the toplevel window.
           top is the toplevel containing window."""
        _bgcolor = '#d9d9d9'  # X11 color: 'gray85'
        _fgcolor = '#000000'  # X11 color: 'black'
        _compcolor = '#d9d9d9'  # X11 color: 'gray85'
        _ana1color = '#d9d9d9'  # X11 color: 'gray85'
        _ana2color = '#ececec'  # Closest X11 color: 'gray92'
        font10 = "-family {Century Gothic} -size 14"
        self.style = ttk.Style()
        if sys.platform == "win32":
            self.style.theme_use('winnative')
        self.style.configure('.', background=_bgcolor)
        self.style.configure('.', foreground=_fgcolor)
        self.style.configure('.', font="TkDefaultFont")
        self.style.map('.', background=
        [('selected', _compcolor), ('active', _ana2color)])

        top.geometry("1920x1001+-101+39")
        top.attributes("-fullscreen", True)
        top.minsize(148, 1)
        top.maxsize(1924, 1055)
        top.resizable(1, 1)
        top.title("New Toplevel")
        top.configure(background="#ed5f83")
        top.configure(highlightbackground="#d9d9d9")
        top.configure(highlightcolor="black")

        self.Label1 = tk.Label(top)
        self.Label1.place(relx=0.340, rely=0.09, height=78, width=549)
        self.Label1.configure(activebackground="#f9f9f9")
        self.Label1.configure(activeforeground="black")
        self.Label1.configure(background="#ed5f83")
        self.Label1.configure(disabledforeground="#a3a3a3")
        self.Label1.configure(font="-family {Trebuchet MS} -size 24 -weight bold")
        self.Label1.configure(foreground="#ffffff")
        self.Label1.configure(highlightbackground="#d9d9d9")
        self.Label1.configure(highlightcolor="black")
        self.Label1.configure(text='''Recommendation of Recipes''')

        self.Button1 = tk.Button(top)
        self.Button1.place(relx=0.922, rely=0.05, height=53, width=68)
        self.Button1.configure(activebackground="#ececec")
        self.Button1.configure(activeforeground="#000000")
        self.Button1.configure(background="#b30000")
        self.Button1.configure(disabledforeground="#a3a3a3")
        self.Button1.configure(font="-family {Segoe UI} -size 14 -weight bold")
        self.Button1.configure(foreground="#ffffff")
        self.Button1.configure(highlightbackground="#d9d9d9")
        self.Button1.configure(highlightcolor="black")
        self.Button1.configure(pady="0")
        self.Button1.configure(text='''X''')
        self.Button1.configure(command=root.destroy)

        meal_list = ['Breakfast', 'Lunch', 'Dinner']
        self.TCombobox1 = ttk.Combobox(top, values=meal_list, state='readonly')
        self.TCombobox1.place(relx=0.135, rely=0.39, relheight=0.036
                              , relwidth=0.129)
        self.TCombobox1.configure(font="-family {Segoe UI} -size 14")
        self.TCombobox1.configure(textvariable=recommendation_support.combobox)
        self.TCombobox1.configure(takefocus="")

        self.Label1_1 = tk.Label(top)
        self.Label1_1.place(relx=0.125, rely=0.34, height=38, width=228)
        self.Label1_1.configure(activebackground="#f9f9f9")
        self.Label1_1.configure(activeforeground="black")
        self.Label1_1.configure(background="#ed5f83")
        self.Label1_1.configure(disabledforeground="#a3a3a3")
        self.Label1_1.configure(font="-family {Trebuchet MS} -size 14")
        self.Label1_1.configure(foreground="#ffffff")
        self.Label1_1.configure(highlightbackground="#d9d9d9")
        self.Label1_1.configure(highlightcolor="black")
        self.Label1_1.configure(text='''Select meal type''')

        self.Label1_2 = tk.Label(top)
        self.Label1_2.place(relx=0.135, rely=0.48, height=39, width=228)
        self.Label1_2.configure(activebackground="#f9f9f9")
        self.Label1_2.configure(activeforeground="black")
        self.Label1_2.configure(background="#ed5f83")
        self.Label1_2.configure(disabledforeground="#a3a3a3")
        self.Label1_2.configure(font="-family {Trebuchet MS} -size 14")
        self.Label1_2.configure(foreground="#ffffff")
        self.Label1_2.configure(highlightbackground="#d9d9d9")
        self.Label1_2.configure(highlightcolor="black")
        self.Label1_2.configure(text='''Enter Ingredients''')

        self.Entry1 = tk.Entry(top)
        self.Entry1.place(relx=0.02, rely=0.529, height=34, relwidth=0.377)
        self.Entry1.configure(background="white")
        self.Entry1.configure(disabledforeground="#a3a3a3")
        self.Entry1.configure(font="-family {Courier New} -size 18")
        self.Entry1.configure(foreground="#000000")
        self.Entry1.configure(highlightbackground="#d9d9d9")
        self.Entry1.configure(highlightcolor="black")
        self.Entry1.configure(insertbackground="black")
        self.Entry1.configure(selectbackground="#c4c4c4")
        self.Entry1.configure(selectforeground="black")

        self.Button2 = tk.Button(top)
        self.Button2.place(relx=0.08, rely=0.609, height=53, width=113)
        self.Button2.configure(activebackground="#ececec")
        self.Button2.configure(activeforeground="#000000")
        self.Button2.configure(background="#b70218")
        self.Button2.configure(disabledforeground="#a3a3a3")
        self.Button2.configure(font="-family {Segoe UI} -size 14 -weight bold")
        self.Button2.configure(foreground="#ffffff")
        self.Button2.configure(highlightbackground="#d9d9d9")
        self.Button2.configure(highlightcolor="black")
        self.Button2.configure(pady="0")
        self.Button2.configure(text='''Submit''')
        self.Button2.configure(command=self.recommend_recipes)

        self.Button2_3 = tk.Button(top)
        self.Button2_3.place(relx=0.17, rely=0.609, height=53, width=290)
        self.Button2_3.configure(activebackground="#ececec")
        self.Button2_3.configure(activeforeground="#000000")
        self.Button2_3.configure(background="#152ba4")
        self.Button2_3.configure(disabledforeground="#a3a3a3")
        self.Button2_3.configure(font="-family {Segoe UI} -size 14 -weight bold")
        self.Button2_3.configure(foreground="#ffffff")
        self.Button2_3.configure(highlightbackground="#d9d9d9")
        self.Button2_3.configure(highlightcolor="black")
        self.Button2_3.configure(pady="0")
        self.Button2_3.configure(text='''Co-occuring Ingredient Search''')
        self.Button2_3.configure(command=self.suggest)

        self.Scrolledlistbox1 = ScrolledListBox(top, selectmode=SINGLE)
        self.Scrolledlistbox1.place(relx=0.51, rely=0.29, relheight=0.546
                                    , relwidth=0.32)
        self.Scrolledlistbox1.configure(background="white")
        self.Scrolledlistbox1.configure(cursor="xterm")
        self.Scrolledlistbox1.configure(disabledforeground="#a3a3a3")
        self.Scrolledlistbox1.configure(font=font10)
        self.Scrolledlistbox1.configure(cursor='hand2')
        self.Scrolledlistbox1.configure(foreground="black")
        self.Scrolledlistbox1.configure(highlightbackground="#d9d9d9")
        self.Scrolledlistbox1.configure(highlightcolor="#d9d9d9")
        self.Scrolledlistbox1.configure(selectbackground="#c4c4c4")
        self.Scrolledlistbox1.configure(selectforeground="black")
        self.Scrolledlistbox1.bind('<<ListboxSelect>>',self.list_item_select)


# The following code is added to facilitate the Scrolled widgets you specified.
class AutoScroll(object):
    '''Configure the scrollbars for a widget.'''

    def __init__(self, master):
        #  Rozen. Added the try-except clauses so that this class
        #  could be used for scrolled entry widget for which vertical
        #  scrolling is not supported. 5/7/14.
        try:
            vsb = ttk.Scrollbar(master, orient='vertical', command=self.yview)
        except:
            pass
        hsb = ttk.Scrollbar(master, orient='horizontal', command=self.xview)
        try:
            self.configure(yscrollcommand=self._autoscroll(vsb))
        except:
            pass
        self.configure(xscrollcommand=self._autoscroll(hsb))
        self.grid(column=0, row=0, sticky='nsew')
        try:
            vsb.grid(column=1, row=0, sticky='ns')
        except:
            pass
        hsb.grid(column=0, row=1, sticky='ew')
        master.grid_columnconfigure(0, weight=1)
        master.grid_rowconfigure(0, weight=1)
        # Copy geometry methods of master  (taken from ScrolledText.py)
        if py3:
            methods = tk.Pack.__dict__.keys() | tk.Grid.__dict__.keys() \
                      | tk.Place.__dict__.keys()
        else:
            methods = tk.Pack.__dict__.keys() + tk.Grid.__dict__.keys() \
                      + tk.Place.__dict__.keys()
        for meth in methods:
            if meth[0] != '_' and meth not in ('config', 'configure'):
                setattr(self, meth, getattr(master, meth))

    @staticmethod
    def _autoscroll(sbar):
        '''Hide and show scrollbar as needed.'''

        def wrapped(first, last):
            first, last = float(first), float(last)
            if first <= 0 and last >= 1:
                sbar.grid_remove()
            else:
                sbar.grid()
            sbar.set(first, last)

        return wrapped

    def __str__(self):
        return str(self.master)


def _create_container(func):
    '''Creates a ttk Frame with a given master, and use this new frame to
    place the scrollbars and the widget.'''

    def wrapped(cls, master, **kw):
        container = ttk.Frame(master)
        container.bind('<Enter>', lambda e: _bound_to_mousewheel(e, container))
        container.bind('<Leave>', lambda e: _unbound_to_mousewheel(e, container))
        return func(cls, container, **kw)

    return wrapped


class ScrolledListBox(AutoScroll, tk.Listbox):
    '''A standard Tkinter Listbox widget with scrollbars that will
    automatically show/hide as needed.'''

    @_create_container
    def __init__(self, master, **kw):
        tk.Listbox.__init__(self, master, **kw)
        AutoScroll.__init__(self, master)

    def size_(self):
        sz = tk.Listbox.size(self)
        return sz


import platform


def _bound_to_mousewheel(event, widget):
    child = widget.winfo_children()[0]
    if platform.system() == 'Windows' or platform.system() == 'Darwin':
        child.bind_all('<MouseWheel>', lambda e: _on_mousewheel(e, child))
        child.bind_all('<Shift-MouseWheel>', lambda e: _on_shiftmouse(e, child))
    else:
        child.bind_all('<Button-4>', lambda e: _on_mousewheel(e, child))
        child.bind_all('<Button-5>', lambda e: _on_mousewheel(e, child))
        child.bind_all('<Shift-Button-4>', lambda e: _on_shiftmouse(e, child))
        child.bind_all('<Shift-Button-5>', lambda e: _on_shiftmouse(e, child))


def _unbound_to_mousewheel(event, widget):
    if platform.system() == 'Windows' or platform.system() == 'Darwin':
        widget.unbind_all('<MouseWheel>')
        widget.unbind_all('<Shift-MouseWheel>')
    else:
        widget.unbind_all('<Button-4>')
        widget.unbind_all('<Button-5>')
        widget.unbind_all('<Shift-Button-4>')
        widget.unbind_all('<Shift-Button-5>')


def _on_mousewheel(event, widget):
    if platform.system() == 'Windows':
        widget.yview_scroll(-1 * int(event.delta / 120), 'units')
    elif platform.system() == 'Darwin':
        widget.yview_scroll(-1 * int(event.delta), 'units')
    else:
        if event.num == 4:
            widget.yview_scroll(-1, 'units')
        elif event.num == 5:
            widget.yview_scroll(1, 'units')


def _on_shiftmouse(event, widget):
    if platform.system() == 'Windows':
        widget.xview_scroll(-1 * int(event.delta / 120), 'units')
    elif platform.system() == 'Darwin':
        widget.xview_scroll(-1 * int(event.delta), 'units')
    else:
        if event.num == 4:
            widget.xview_scroll(-1, 'units')
        elif event.num == 5:
            widget.xview_scroll(1, 'units')


**Main screen for creating and invoking GUI screen**

In [ ]:
if __name__ == '__main__':
    vp_start_gui()